In [39]:
%load_ext autoreload
%autoreload 2

import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


**REQUIREMENTS**

Python >=3.8


### 1 - create a SBM graph and extract the edges list


In [40]:
from os.path import join as pjoin
from fury import window, colormap as cmap
import numpy as np
from fury.data.fetcher import fetch_viz_wiki_nw

files, folder = fetch_viz_wiki_nw()
categories_file, edges_file, positions_file = sorted(files.keys())

###############################################################################
# We read our datasets


categories = np.loadtxt(pjoin(folder, categories_file), dtype=str)
edges_list = np.loadtxt(pjoin(folder, edges_file), dtype=int)
num_nodes = len(categories)

positions = np.random.normal(size=(num_nodes, 3))
###############################################################################
# We attribute a color to each category of our dataset which correspond to our
# nodes colors.

category2index = {category: i
                  for i, category in enumerate(np.unique(categories))}


index2category = np.unique(categories)

categoryColors = cmap.distinguishable_colormap(nb_colors=len(index2category))

categoryMarkers = ['o', 's', 'd']

colors = np.array([categoryColors[category2index[category]]
                   for category in categories])

markers = [categoryMarkers[category2index[category]]
                   for category in categories]

Dataset is already in place. If you want to fetch it again please first remove the folder /home/devmessias/.fury/examples/wiki_nw 
More information about complex networks can be found in this papers: https://arxiv.org/abs/0711.3199


In [41]:
edgesColors = []
for source, target in edges_list:
    edgesColors.append(np.array([colors[source], colors[target]]))

edgesColors = np.average(np.array(edgesColors), axis=1)

### 2 - Creates a network superactor and adds to a FURY Scene

In [42]:
from helios.fury.actors import FurySuperActorNetwork
from fury import window


network_super_actor = FurySuperActorNetwork(
        positions=positions,
        colors=colors,
        scales=1,
        node_edge_width=0,
        marker=markers,
        edges=edges_list,
        edge_line_color=edgesColors
)


scene = window.Scene()
scene.add(network_super_actor.nodes.vtk_actor)
scene.add(network_super_actor.edges.vtk_actor)

### 3 - Creates a ShowManager FURY Obj.

In [43]:
from fury import actor, window

window_size = [400, 400]
showm = window.ShowManager(scene, reset_camera=False, size=(
    window_size[0], window_size[1]), order_transparent=False,
)

### 4- Starts the streaming

In [44]:
from fury.stream.widget import Widget

widget = Widget(showm, encoding='webrtc')
#widget = Widget(showm)
widget.start()

created 20 8 1344
url: http://localhost:8410?iframe=1&encoding=webrtc


In [46]:
widget.return_iframe(300)

### 5 - Start Force-Directed async layout

- ms equals to 100 means that the vtk_positions will be updated at each 100 ms

In [55]:
from helios.core.layout import HeliosFr
layout = HeliosFr(positions,edges_list,showm, network_super_actor, update_interval_workers=50)
print('start async helios')
# if you want to disable the auto-update position chose a ms <0
ms = 60
layout.start(ms)

start async helios


In [63]:
# if you passed ms<0  execute this cell
layout.update_in_vtk()

### 6 - Change some network properties

In [54]:
network_super_actor.nodes.marker_opacity = .5
showm.window.Render()
network_super_actor.nodes.edge_width = .1
network_super_actor.nodes.Uniforms.edgeColor.value = (255, 255, 255)
showm.window.Render()

In [64]:
layout.stop()

In [38]:
widget.stop()